### The usual Kaggle thingy

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#! pip install deepspeed mpi4py

### Install `aitextgen` package

In [ ]:
!pip install -q aitextgen #install the main package

### Load `aitextgen`

In [ ]:
from aitextgen import aitextgen

### Know your GPU Config 

In [ ]:
! nvidia-smi

### Download the 124M GPT2 Model

In [ ]:
ai = aitextgen(tf_gpt2="124M", to_gpu=True)


### Read the Input Dataset

In [ ]:
input_file = pd.read_csv("../input/6000-indian-food-recipes-dataset/IndianFoodDatasetCSV.csv")

### Avoid Pandas truncating strings - Remove any column width limit

In [ ]:
pd.set_option('display.max_colwidth', None)

### Text Cleaning


In [ ]:
input_file["TranslatedRecipeName"]  = input_file["TranslatedRecipeName"].str.replace('Recipe','').str.replace('(','').str.replace(')','')

### Assign the output to a new DataFrame

In [ ]:
df = pd.DataFrame(input_file["TranslatedRecipeName"])

In [ ]:
df.shape

### More Cleaning

In [ ]:
df = df.assign(var1=df['TranslatedRecipeName'].str.split('-')).explode('var1')


In [ ]:
df.var1 = df.var1.str.lstrip()

In [ ]:
df.shape

### Save the text (only that column) as a text file 

In [ ]:
df.to_csv("input_text_cleaned.txt", columns=["var1"], header=False, index=False)

In [ ]:
#from aitextgen.TokenDataset import TokenDataset

In [ ]:
#data = TokenDataset('./translatedrecipes.csv', line_by_line=True)

### Use the above saved text file for fine-tuning - set the right parameters 

In [ ]:
ai.train('input_text_cleaned.txt',
         line_by_line=False,
         from_cache=False,
         num_steps=500,
         generate_every=100,
         save_every=500,
         save_gdrive=False,
         learning_rate=1e-3,
         fp16=False,
         batch_size=1, 
         )

### Load the newly fine-tuned model which is saved in `trained_model` directory

In [ ]:
ai = aitextgen(model="./trained_model/pytorch_model.bin", config="./trained_model/config.json", to_gpu=True)

### Here you go 🎉 AI Generated Text

In [ ]:
ai.generate(n=3,
            batch_size=1,
            max_length=50,
            temperature=1.0,
            top_p=0.9)